In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pdb
import rospy
import numpy as np
import torch
from visualization_msgs.msg import *

# Adds parent dir to path
current_dir = os.getcwd()
base_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, base_dir)

from Models.DiscreteBKI import *
from Data.dataset import Rellis3dDataset, ray_trace_batch
from Data.utils import *
from model_utils import *
from Benchmarks.eval_utils import *
from sklearn.metrics import jaccard_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EVAL_SEGMENTATION=True
EVAL_SSC=False
EVAL_GEOMETRIC=False
TIME_MEASUREMENTS=False
NUM_CLASSES=21
DATASET_DIR = "/home/arthurzhang/Data/Rellis-3D"
sbki_folder = "/home/arthurzhang/CURLY/Baselines/catkin_ws/src/BKISemanticMapping/data/rellis3d/test"


In [ ]:
idx=10
ssc_evaluation_dir = os.path.join(sbki_folder, "ssc", str(idx).zfill(6)+".txt")
ssc_evaluation = np.fromfile(ssc_evaluation_dir, sep="\n",dtype=np.uint8).reshape(256, 256, 16)

sem_evaluation_dir = os.path.join(sbki_folder, "evaluations", str(idx).zfill(6)+".txt")
sem_evaluation = np.fromfile(sem_evaluation_dir, sep="\n",dtype=np.uint8)

#Run for visualizations
rospy.init_node('talker', anonymous=True)
map_pub = rospy.Publisher('SemMap', MarkerArray, queue_size=10)
bki_map = DiscreteBKI(
    torch.tensor([256, 256, 16]).to(device), # Grid size
    torch.tensor([-25.6, -25.6, -2.0]).to(device), # Lower bound
    torch.tensor([25.6, 25.6, 1.2]).to(device), # Upper bound
    device=device
)

# ssc_map = torch.from_numpy(ssc_evaluation)
# publish_voxels(ssc_map, map_pub, bki_map.centroids,
#     bki_map.min_bound.reshape(-1),
#     bki_map.max_bound.reshape(-1),
#     bki_map.grid_size.reshape(-1))

rellis_ds = Rellis3dDataset(directory=DATASET_DIR, device=device, 
    num_frames=10, remap=True, use_aug=False, use_gt=True, model_setting="test")
print(len(rellis_ds))

gt_pc, gt_labels, _, _, _ = rellis_ds[idx]
# pdb.set_trace()
# publish_voxels(torch.from_numpy(gt_voxels_np), map_pub, bki_map.centroids,
#     bki_map.min_bound.reshape(-1),
#     bki_map.max_bound.reshape(-1),
#     bki_map.grid_size.reshape(-1))

gt_pc = np.vstack(np.array(gt_pc))
gt_labels = np.vstack(np.array(gt_labels)).reshape(-1)

gt_pc = torch.from_numpy(gt_pc)
gt_labels = torch.from_numpy(gt_labels)
sem_map = torch.from_numpy(sem_evaluation)
pdb.set_trace()
publish_pc(gt_pc, sem_map, map_pub,
    bki_map.min_bound.reshape(-1),
    bki_map.max_bound.reshape(-1),
    bki_map.grid_size.reshape(-1), use_mask=False)
pdb.set_trace()
publish_pc(gt_pc, gt_labels, map_pub,
    bki_map.min_bound.reshape(-1),
    bki_map.max_bound.reshape(-1),
    bki_map.grid_size.reshape(-1), use_mask=False)

    

In [6]:
gt_all = np.array([])
pred_all = np.array([])

rellis_ds = Rellis3dDataset(directory=DATASET_DIR, device=device, 
    num_frames=10, remap=True, use_aug=False, use_gt=True, do_ssc=False, model_setting="test")


num_correct = 0
num_total = 0
# running_inter = np.zeros((NUM_CLASSES,))
# running_union =  np.zeros((NUM_CLASSES,))
running_inter = np.zeros((NUM_CLASSES,))
running_union = np.zeros((NUM_CLASSES,)) + 1e-6
running_ssc_inter = np.zeros(NUM_CLASSES,)
running_ssc_union = np.zeros(NUM_CLASSES,) + 1e-6
running_geo_score = np.zeros((2,))
running_counter = 0
curr_time = time.time()

if EVAL_SEGMENTATION:
    for idx in range(0, len(rellis_ds)):
        sem_evaluation_dir = os.path.join(sbki_folder, "evaluations", str(idx).zfill(6)+".txt")

        # Load rellis3D dataset into np arrays
        pc, gt_labels, gt_voxels_np, _, _ = rellis_ds[idx]
        gt_labels_np= np.array(gt_labels[-1]).reshape(-1)
        pc_np       = np.array(pc[-1])

        total_num_points= np.vstack(np.array(gt_labels)).shape[0]
        curr_frame_size = gt_labels[-1].shape[0]
        offset_amount   = total_num_points - curr_frame_size

        if EVAL_SEGMENTATION:
            sem_evaluation = np.loadtxt(sem_evaluation_dir, skiprows=offset_amount,
                    delimiter='\n', dtype= np.uint8)

        sem_evaluation_torch = torch.from_numpy(sem_evaluation).to(device=device)
        gt_labels_torch      = torch.from_numpy(gt_labels_np).to(device=device)
       
        inter, union = iou_one_frame(sem_evaluation_torch, gt_labels_torch, n_classes=NUM_CLASSES)
        running_inter   += inter
        running_union   += union

        num_correct += np.sum(sem_evaluation==gt_labels_np)
        num_total   += gt_labels_np.shape[0]

        # running_union[running_union==0] = 1
        # running_iou = (running_inter / running_union)
        if running_counter%100==0:
            print("mIoU ", np.mean(running_inter/running_union))
            print("Accuracy ", num_correct / num_total)
            print("Time for iteration ", running_counter, " is ", time.time() - curr_time)
        running_counter += 1

        if TIME_MEASUREMENTS:
            print("Time to complete loop ", time.time() - curr_time)
            curr_time = time.time()
print("Final mIoU ", np.mean(running_inter / running_union))
print("Final IoU ", running_inter / running_union)
print("Final Accuracy ", num_correct / num_total)

mIoU  0.10810891860498886
Accuracy  0.7149382507189985
Time for iteration  0  is  0.0623784065246582
mIoU  0.1272309502756785
Accuracy  0.7384191622036109
Time for iteration  100  is  25.497965335845947
mIoU  0.11412061490501815
Accuracy  0.6731991915738398
Time for iteration  200  is  51.42003798484802
mIoU  0.1329589779560917
Accuracy  0.6943877167404007
Time for iteration  300  is  80.4188323020935
mIoU  0.12921645489282502
Accuracy  0.6738377478428141
Time for iteration  400  is  106.63422012329102
mIoU  0.13232312990341635
Accuracy  0.6906800610500419
Time for iteration  500  is  135.15677165985107
mIoU  0.13542862551651857
Accuracy  0.7142231200441462
Time for iteration  600  is  163.1722228527069
mIoU  0.13184898671460124
Accuracy  0.7034944354940565
Time for iteration  700  is  190.68327403068542
mIoU  0.1305210362186075
Accuracy  0.6975747353823779
Time for iteration  800  is  217.96564888954163
mIoU  0.12964984144561445
Accuracy  0.6976770296636122
Time for iteration  900  is

In [ ]:
if EVAL_SSC:
    for idx in range(0, 80):
        sem_evaluation_dir = os.path.join(sbki_folder, "evaluations", str(idx).zfill(6)+".txt")
        ssc_evaluation_dir = os.path.join(sbki_folder, "ssc", str(idx).zfill(6)+".txt")
        
        if TIME_MEASUREMENTS: curr_time = time.time()

        if EVAL_SEGMENTATION:
            sem_evaluation = np.fromfile(sem_evaluation_dir, sep="\n",dtype=np.uint8)
        if EVAL_SSC:
            ssc_evaluation = np.fromfile(ssc_evaluation_dir, sep="\n",dtype=np.uint8)

        if TIME_MEASUREMENTS: 
            print("time to load files ", time.time() - curr_time)
            curr_time = time.time()

        # Load rellis3D dataset into np arrays
        pc, gt_labels, gt_voxels_np, _, _ = rellis_ds[idx]
        gt_labels_np= np.vstack(np.array(gt_labels))
        pc_np       = np.vstack(np.array(pc))

        if TIME_MEASUREMENTS:
            print("time to load rellis ", time.time() - curr_time)
            curr_time = time.time()

        # Process free space labels
        if EVAL_SSC:
            fs_pc           = ray_trace_batch(pc_np, gt_labels_np, 0.3, device)
            pc_np           = np.vstack( (pc_np, fs_pc[:, :3].reshape(-1, 3)))
        else:
            pc_np    = pc_np.reshape(-1, 3)

        if TIME_MEASUREMENTS:
            print("time to raytrace ", time.time() - curr_time)
            curr_time = time.time()
        
        if pc_np.shape[0] <= 0:
            # Zero pad in case all labels are 0
            pc_np = np.zeros((1, 3))
            gt_labels_np = np.zeros((1,))

        # Perform Evaluations
        if EVAL_SEGMENTATION:
            sem_evaluation_torch = torch.from_numpy(sem_evaluation)
            gt_labels_torch      = torch.from_numpy(gt_labels_np)

            inter, union = iou_one_frame(sem_evaluation_torch, gt_labels_torch, n_classes=NUM_CLASSES)
            running_inter   += inter
            running_union   += union

        # Visualization for Debugging
        # gt_pc_torch = torch.from_numpy(gt_pc_np)
        # preds = torch.from_numpy(sem_evaluation.astype(np.uint8)).reshape(-1)
        # publish_pc(gt_pc_torch, preds, map_pub, 
        #         bki_map.min_bound.reshape(-1),
        #         bki_map.max_bound.reshape(-1),
        #         bki_map.grid_size.reshape(-1))
        # pdb.set_trace()
        # gt_labels_torch = torch.from_numpy(gt_labels_np.astype(np.uint8)).reshape(-1)
        # publish_pc(gt_pc_torch, gt_labels_torch, map_pub, 
        #         bki_map.min_bound.reshape(-1),
        #         bki_map.max_bound.reshape(-1),
        #         bki_map.grid_size.reshape(-1))
        # pdb.set_trace()

        if EVAL_SSC:
            gt_voxels_np   = gt_voxels_np.reshape(-1)
            ssc_evaluation_torch = torch.from_numpy(ssc_evaluation)
            gt_voxels_torch      = torch.from_numpy(gt_voxels_np)
            ssc_inter, ssc_union = iou_one_frame(ssc_evaluation_torch, gt_voxels_torch, n_classes=NUM_CLASSES)
            running_ssc_inter   += ssc_inter
            running_ssc_union   += ssc_union
        
        if TIME_MEASUREMENTS:
            print("time to compute iou ", time.time() - curr_time)

        if EVAL_GEOMETRIC:
            _, _, iou = geo_complete_score(gt_voxels_np, ssc_evaluation, LABELS_REMAP[-1])
            running_geo_score += iou

        running_counter += 1
        num_correct += np.sum(sem_evaluation==gt_labels_np)
        num_total   += gt_labels_np.shape[0]

        # running_union[running_union==0] = 1
        # running_iou = (running_inter / running_union)
        if running_counter%40==0:
            if EVAL_SEGMENTATION:
                print("mIoU ", np.mean(running_inter/running_union))
            if EVAL_SSC:
                print("SSC mIoU ", np.mean(running_ssc_inter / running_ssc_union))
            if EVAL_GEOMETRIC:
                print("Geometric Completeness ", (running_geo_score / running_counter))
            print("Accuracy ", num_correct / num_total)
            print("Time for iteration ", running_counter, " is ", time.time() - curr_time)
        
        curr_time = time.time()

    print("Final mIoU ", np.mean(running_inter / running_union))
    print("Final SSC mIoU ", np.mean(running_ssc_inter / running_ssc_union))
    print("Final IoU ", running_inter / running_union)
    print("Final SSC IoU ", running_ssc_inter / running_ssc_union)
    print("Final Geometric Completeness ", (running_geo_score / running_counter))
    print("Final Accuracy ", num_correct / num_total)


In [7]:
running_inter = running_inter[running_union!=1e-6]
running_union = running_union[running_union!=1e-6]

print("Final mIoU ", np.mean(running_inter / running_union))
# print("Final SSC mIoU ", np.mean(running_ssc_iou / running_counter))
print("Final IoU ", running_inter / running_union)
# print("Final SSC IoU ", running_ssc_iou / running_counter)
# print("Final Geometric Completeness ", (running_geo_score / running_counter))
print("Final Accuracy ", num_correct / num_total)

Final mIoU  0.28633979633710843
Final IoU  [0.         0.65096859 0.57483074 0.40543317 0.         0.
 0.         0.         0.19299055 0.70675698 0.73473916 0.68889371
 0.24145292 0.09903114 0.        ]
Final Accuracy  0.7922276771026712


In [ ]:

final_iou_scores = np.array([
    0.00000000e+00,
    0.00000000e+00,
    5.60016396e-01,
    5.88545616e-01,
    1.43518530e-01,
    9.22742464e-06,
    0.00000000e+00,
    0.00000000e+00,
    0.00000000e+00,
    2.62998573e-03,
    0.00000000e+00,
    0.00000000e+00,
    0.00000000e+00,
    6.06223123e-02,
    5.21189291e-01,
    3.04980999e-01,
    7.74924694e-02,
    1.03251444e-01,
    1.13727678e-01,
    2.97595047e-03,
    9.56348479e-01])

final_ssc_iou_scores = np.array([
    6.70416552e-01,
    0.00000000e+00, 
    8.29817942e-03, 
    5.21273223e-04,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    0.00000000e+00, 
    7.47226060e-06, 
    1.41210174e-03, 
    1.08312520e-03,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    5.60834131e-02
])

# print("Number correct ", num_correct, " number total ", num_total)
print("mIOU with empty classes removed ", np.mean(final_iou_scores[CLASS_COUNTS_REMAPPED>0]))
print("ssc IOU with empty classes removed ", np.mean(final_ssc_iou_scores[CLASS_COUNTS_REMAPPED>0]))